# 오차 역전파 - Backpropagation
[Backpropagation](https://wikidocs.net/37406)

X - w -> Y - J -> Y'  
w: Weight  
J: Loss Function  
오차역전파는 dJ/dw가 0에 가까워지는 w, 즉, J를 가장 작게 만들어줄 w를 찾는 것이다. w를 찾았으면, 그 이전 노드 단계에서의 w를 찾고, 찾았으면 그 이전의 것을 찾고... 반복하는 것이다.  

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 계산 그래프 - Computational graph
- 노드(node)
- 에지(edge)

### 1.1 계산 그래프 예제

#### 문제1) 1개에 100원인 사과를 2개 샀다. 단 소비세는 10%이다.

- 계산 그래프 적용
<img src="./images/fig_5-1.png" width="500"/>

- 사과의 개수와 소비세를 변수로 설정
<img src="./images/fig_5-2.png" width="500"/>

#### 문제2) 1개에 100원인 사과를 2개, 1개에 150원인 귤 3개를 샀다. 단 소비세는 10%이다.

- 계산 그래프 적용
<img src="./images/fig_5-3.png" width="500"/>

### 1.2 계산 그래프의 특징

- 중간 계산 결과 보관 가능

- 국소적 계산
<img src="./images/fig_5-4.png" width="500"/>

- 역전파에 의한 미분 값의 전달(미분을 효율적으로 계산)
<img src="./images/fig_5-5.png" width="500"/>

# 2 연쇄법칙 - Chain rule

### 2.1 계산 그래프의 역전파

- $y = f(x)$
<img src="./images/fig_5-6.png" width="300"/>

### 2.2 합성 함수의 미분

- $z = (x + y)^2$

- 합성 함수의 분해 - 치환
<img src="./images/e_5.1.png" width="100"/>

- 연쇄법칙(Chain rule) 적용
<img src="./images/e_5.2.png" width="150"/>

- 각 함수 미분
<img src="./images/e_5.3.png" width="100"/>

- 합성 함수 미분 계산 - 치환 변수 환원( $t = x + y$ )
<img src="./images/e_5.4.png" width="300"/>

### 2.3 연쇄법칙과 계산 그래프

- 연쇄법칙을 계산 그래프에 적용
<img src="./images/fig_5-7.png" width="400"/>

- 연쇄법칙과 계산 그래프를 통해 합성 함수의 미분 구현
- $z = (x + y)^2$
<img src="./images/fig_5-8.png" width="400"/>

# 3 역전파

### 3.1 덧셈 노드의 역전파

- 국소적 계산
<img src="./images/fig_5-10.png" width="500"/>

#### 3.2.1 덧셈 노드의 역전파 구현

- 함수
$$z = x + y$$

- 함수의 편미분
<img src="./images/e_5.5.png" width="70"/>

- 덧셈 노드의 순전파와 역전파: *덧셈 노드의 역전파는 입력값을 그대로 흘려 보낸다.*
<img src="./images/fig_5-9.png" width="500"/>

- 덧셈 노드의 구체적인 예
<img src="./images/fig_5-11.png" width="500"/>

### 3.2 곱셈 노드의 역전파

- 함수
$$z = xy$$

- 함수의 편미분
<img src="./images/e_5.6.png" width="70"/>

- 곱셈 노드의 순전파와 역전파: *곱셈 노드의 역전파는 입력값을 서로 바꾸고 곱해서 흘려 보낸다.*
<img src="./images/fig_5-12.png" width="500"/>

- 곱셈 노드의 구체적인 예
<img src="./images/fig_5-13.png" width="500"/>

### 3.3 사과 쇼핑의 예

- 사과 쇼핑의 역전파 예
<img src="./images/fig_5-14.png" width="500"/>

- 사과와 귤 쇼핑의 역전파 예 - 문제
<img src="./images/fig_5-15.png" width="500"/>

- 사과와 귤 쇼핑의 역전파 예 - 정답
<img src="./images/fig_5-17.png" width="500"/>

# 4 연산 계층 코드 구현
- AddLayer
- MulLayer

### 4.1 덧셈 계층 코드 구현

In [ ]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### 4.2 곱셈 계층 코드 구현

In [ ]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

### 4.3 사과 쇼핑 계층 실행

- 사과 2개 구입
<img src="./images/fig_5-16.png" width="500"/>

In [ ]:
# 입력값 설정
apple = 100
apple_num = 2

tax = 1.1

In [ ]:
# 곱셈 계층 생성
mul_apple_layer = MulLayer()
mul_tax_layer   = MulLayer()

In [ ]:
# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price       = mul_tax_layer.forward(apple_price, tax)

In [ ]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [ ]:
# 출력
print('price:', int(price))
print('dApple:', dapple)
print('dApple_num:', int(dapple_num))
print('dTax:', dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


### 4.4 사과, 귤 쇼핑 계층 실행

- 사과 2개, 귤 3개 구입
<img src="./images/fig_5-17.png" width="500"/>

In [ ]:
# 입력값 설정
apple = 100
apple_num = 2

orange = 150
orange_num = 3

tax = 1.1

In [ ]:
# 곱셈 계층, 덧셈 계층 생성
mul_apple_layer  = MulLayer()
mul_orange_layer = MulLayer()

add_apple_orange_layer = AddLayer()

mul_tax_layer = MulLayer()

In [ ]:
# forward
apple_price  = mul_apple_layer.forward(apple, apple_num)               # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)            # (2)

all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price     = mul_tax_layer.forward(all_price, tax)                      # (4)

In [ ]:
# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)                          # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)            # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)                # (1)

In [ ]:
# 출력
print('price:', int(price))

print('dApple:', dapple)
print('dApple_num:', int(dapple_num))
print('dOrange:', dorange)
print('dOrange_num:', int(dorange_num))
print('dTax:', dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# 5 활성화 함수 계층 코드 구현
- ReLU
- Sigmoid

### 5.1 ReLU 계층 코드 구현

- ReLU 함수 수식
<img src="./images/e_5.7.png" width="150"/>

- ReLU 함수 미분
<img src="./images/e_5.8.png" width="150"/>

- ReLU 계층의 계산 그래프
<img src="./images/fig_5-18.png" width="500"/>

#### ReLU 계층 코드 구현

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

#### <참고> mask 활용 예제

In [ ]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [ ]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


In [ ]:
x[mask] = 0
print(x)

[[1. 0.]
 [0. 3.]]


### 5.2 Sigmoid 계층 코드 구현

- Sigmoid 함수 수식
<img src="./images/e_5.9.png" width="200"/>

- Sigmoid 계층의 계산 그래프(순전파)
<img src="./images/fig_5-19.png" width="600"/>

- Sigmoid 계층의 계산 그래프(역전파)
<img src="./images/fig_5-20.png" width="600"/>

- Sigmoid 계층의 계산 그래프(간소화 버전)
<img src="./images/fig_5-21.png" width="300"/>

- Sigmoid 함수 미분
<img src="./images/e_5.12.png" width="400"/>

- Sigmoid 계층의 계산 그래프: 순전파의 출력 y 만으로 계산 가능
<img src="./images/fig_5-22.png" width="300"/>

#### Sigmoid 계층 코드 구현

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out    # 순전파의 출력 보관 후 역전파때 사용
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

# 6 Affine/Softmax 계층 코드 구현

### 6.1 Affine 계층 코드 구현

- Affine 계층의 계산 그래프(순전파): 변수가 행렬
<img src="./images/fig_5-24.png" width="300"/>

- Affine 계층 미분
<img src="./images/e_5.13.png" width="150"/>

- 행렬과 전치 행렬
<img src="./images/e_5.14.png" width="150"/>

- Affine 계층의 계산 그래프(역전파): 변수가 행렬
<img src="./images/fig_5-25.png" width="500"/>

- 변수의 형상
<img src="./images/e_5.15.png" width="300"/>

- 배치용 Affine 계층의 계산 그래프
<img src="./images/fig_5-27.png" width="500"/>

#### Affine 계층 코드 구현

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### 6.2 Softmax 계층 코드 구현

- Softmax 계층의 출력
<img src="./images/fig_5-28.png" width="500"/>

- Softmax-with-Loss 계층의 계산 그래프
<img src="./images/fig_5-30.png" width="400"/>

#### Softmax-with-Loss 계층 코드 구현

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

# 7 오차 역전파 코드 구현

### 7.1 오차 역전파를 적용한 신경망 코드 구현

In [ ]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 7.2 수치 미분 - 오차 역전파 기울기 비교

In [ ]:
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [ ]:
%time
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

# 수치 미분 사용
grad_numerical = network.numerical_gradient(x_batch, t_batch)

# 오차 역전파 사용
grad_backprop  = network.gradient(x_batch, t_batch)

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ':' + str(diff))

W1:4.872482638427244e-10
b1:2.9889343011897923e-09
W2:5.904457854753626e-09
b2:1.3932372812663908e-07


### 7.3 오차 역전파를 사용한 신경망 학습 코드 구현

In [ ]:
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [ ]:
# 입력값 설정
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

In [ ]:
%%time
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

# 네트워크 생성
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f'<{int(i//iter_per_epoch):03}> train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

<000> train acc: 0.117, test acc: 0.118
<001> train acc: 0.904, test acc: 0.909
<002> train acc: 0.923, test acc: 0.923
<003> train acc: 0.935, test acc: 0.936
<004> train acc: 0.945, test acc: 0.943
<005> train acc: 0.952, test acc: 0.951
<006> train acc: 0.956, test acc: 0.955
<007> train acc: 0.961, test acc: 0.959
<008> train acc: 0.964, test acc: 0.960
<009> train acc: 0.968, test acc: 0.965
<010> train acc: 0.970, test acc: 0.966
<011> train acc: 0.971, test acc: 0.967
<012> train acc: 0.974, test acc: 0.967
<013> train acc: 0.974, test acc: 0.966
<014> train acc: 0.975, test acc: 0.968
<015> train acc: 0.978, test acc: 0.972
<016> train acc: 0.979, test acc: 0.971
CPU times: total: 45.2 s
Wall time: 35.4 s


# 정리

- 계산그래프를 이용하연 계산 과정을 시각적으로 파악할수 있다.
- 계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다.
- 계산 그래프의 순전파는 통상의 계산을 수행한다. 한편, 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다.
- 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할수 있다(오차역전파법).
- 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다(기울기 확인).

---

# NOTE  
   
[Backpropagation](https://wikidocs.net/37406)  
  
- 사과 쇼핑의 역전파 예
<img src="./images/fig_5-14.png" width="500"/>  
  
- 연쇄법칙(Chain rule) 적용
<img src="./images/e_5.2.png" width="150"/>  
  
- 각 함수 미분
<img src="./images/e_5.3.png" width="100"/>  
  
- 합성 함수 미분 계산 - 치환 변수 환원( $t = x + y$ )
<img src="./images/e_5.4.png" width="300"/>  
  
---  
  
Input Layer -> Hidden Layer -> Output Layer => Y  
  
Input ~ Hidden에서 w1가 쓰이고, Hidden ~ Output에서 w2가 쓰인다고 하자.  
또한, Cost Function은 t-y1, t-y2, ..., t-y100을 전부 이용해서 만든 잔차의 함수이다.(t가 정답, y가 예측값)  
  
y값을 찾는 과정을 추론(Inference, Predict) 혹은 순전파라고 하며, 반대로 거슬러 올라가 w값을 찾는 것을 학습(Training) 혹은 역전파라고 한다.  
  
원래는 역전파를 안 쓰고, 위에서 소개한 연쇄법칙(Chain rule)을 이용하여 dC/dw(w값 변화에 따른 최종 결과값 Cost Function의 변화량을 의미한다.)를 구하면 됐다.  
하지만, dC/dw를 구하기까지 chain에 속해있는 dC/dA, dA/dB, dB/dD, dD/dw들이 문제였는데..  
  
dC/dA는 A로 표현이 되고, dA/dB는 B로 표현이 되고, dB/dD는 D로 표현이 되고, dD/dw는 w로 표현이 된다.  
우리는 w로 표현된 C를 구해야 하니까, w와 A, B, D간의 관계식을 통해 각각 치환하여 모두 w로 표현하는 과정이 필요했다.  
  
하지만 이 치환이 현실적으로 불가능하다. 식이 너무 많다.  
  
---
  
그래서 우리는 아래의 방법 (오차역전파)를 사용하기로 했다.  
  
우리가 관심있는 것은 w값의 변화에 따른 C값의 변화량인 dC/dw이다.  
왜냐? dC/dw가 가장 작게 나오는 w값이 우리가 찾는 최적의 w값임을 증명하기 때문이다!  
  
우선 임의의 w1을 이용해서 D를 구한다. D로 B를 구하고, B로 A를 구한 뒤, A로 C를 구한다. 즉, 순전파를 한다.  
C를 구했으면, C에서 A로 가는 변화량(기울기)를 구한다. 또, 역으로 A에서 B로의 변화량인 기울기를 구하고, B에서 D로의 기울기, D에서 w로의 기울기를 구한다.  
  
이 기울기들을 chain하면 최종적으로 dC/dw를 알 수 있다.  
원한다면, dC/dB, dC/dA들도 알 수 있다.(chain을 중간에 끊으면 됨.)  
  
단, 우리가 방금 구한 dC/dw는 w1에 대해서만 옳은 값이다.  
즉, w2에 대해서는 다시 순전파해서 값들 싹 구하고, 역으로 올라오면서 기울기 다 구해서 chain해서 dC/dw 구해야 한다.  
매우 귀찮지만, 적어도 치환의 어려움과 복잡성으로 사실상 계산이 복잡한 기존의 방식보다는 컴퓨터가 좋아한다.   
  
이를 오차역전파라고 한다.  

### 4.1 덧셈 계층 코드 구현

In [ ]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### 4.2 곱셈 계층 코드 구현

In [ ]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

### 5.1 ReLU 계층 코드 구현

- ReLU 함수 수식
<img src="./images/e_5.7.png" width="150"/>

- ReLU 함수 미분
<img src="./images/e_5.8.png" width="150"/>

- ReLU 계층의 계산 그래프
<img src="./images/fig_5-18.png" width="500"/>

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

### 5.2 Sigmoid 계층 코드 구현

- Sigmoid 함수 수식
<img src="./images/e_5.9.png" width="200"/>

- Sigmoid 계층의 계산 그래프(순전파)
<img src="./images/fig_5-19.png" width="600"/>

- Sigmoid 계층의 계산 그래프(역전파)
<img src="./images/fig_5-20.png" width="600"/>

- Sigmoid 계층의 계산 그래프(간소화 버전)
<img src="./images/fig_5-21.png" width="300"/>

- Sigmoid 함수 미분
<img src="./images/e_5.12.png" width="400"/>

- Sigmoid 계층의 계산 그래프: 순전파의 출력 y 만으로 계산 가능
<img src="./images/fig_5-22.png" width="300"/>

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out    # 순전파의 출력 보관 후 역전파때 사용
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

### 6.2 Softmax 계층 코드 구현

- Softmax 계층의 출력
<img src="./images/fig_5-28.png" width="500"/>

- Softmax-with-Loss 계층의 계산 그래프
<img src="./images/fig_5-30.png" width="400"/>  

Softmax with Loss 계층의 계산 그래프에서 `y - t`는 손실(오차)을 나타냅니다. 이 계층은 주로 분류 문제에서 사용되며, 입력으로 들어온 데이터를 확률로 변환한 후 이 확률과 실제 정답과의 차이를 계산합니다.

Softmax with Loss 계층의 구성은 다음과 같습니다:

1. **Softmax 계층:** 입력 데이터를 확률 분포로 변환합니다. 즉, 주어진 입력에 대한 각 클래스의 확률을 계산합니다.

2. **Cross-Entropy Loss (교차 엔트로피 손실):** Softmax에서 얻은 확률 분포와 실제 정답 분포(원-핫 인코딩된 레이블) 간의 차이를 측정합니다. 이는 모델의 예측이 실제와 얼마나 다른지를 나타냅니다.

이때 `y`는 Softmax 계층의 출력(모델의 예측), `t`는 실제 정답 분포(레이블)입니다. 따라서 `y - t`는 예측과 실제 간의 차이를 나타내며, 이 차이가 작을수록 손실은 작아지고 모델의 성능이 좋다고 판단됩니다. 이러한 손실을 최소화하면서 모델을 학습시키는 것이 분류 문제에서의 주요 목표입니다.

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

### 7.1 오차 역전파를 적용한 신경망 코드 구현

In [ ]:
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 7.3 오차 역전파를 사용한 신경망 학습 코드 구현

In [ ]:
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [ ]:
# 입력값 설정
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

In [ ]:
%%time
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

# 네트워크 생성
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10) #TwoLayerNet은 위에서 미리 만들어둔 클래스이다.

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f'<{int(i//iter_per_epoch):03}> train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

<000> train acc: 0.139, test acc: 0.141
<001> train acc: 0.904, test acc: 0.909
<002> train acc: 0.924, test acc: 0.927
<003> train acc: 0.935, test acc: 0.935
<004> train acc: 0.945, test acc: 0.945
<005> train acc: 0.952, test acc: 0.951
<006> train acc: 0.957, test acc: 0.953
<007> train acc: 0.961, test acc: 0.957
<008> train acc: 0.965, test acc: 0.959
<009> train acc: 0.968, test acc: 0.961
<010> train acc: 0.970, test acc: 0.964
<011> train acc: 0.972, test acc: 0.966
<012> train acc: 0.974, test acc: 0.967
<013> train acc: 0.975, test acc: 0.967
<014> train acc: 0.976, test acc: 0.967
<015> train acc: 0.978, test acc: 0.968
<016> train acc: 0.978, test acc: 0.970
CPU times: total: 49 s
Wall time: 39.3 s


In [ ]:
# End of file